In [1]:
import pandas as pd
import numpy as np
from deap import base
from deap import creator
from deap import tools
from deap import algorithms

In [2]:
# espaços do caminhão/problema
limite = 2

In [3]:
produtos = [
    {"nome": "Geladeira", "espaco": 0.751, "valor": 999.90},
    {"nome": "Celular", "espaco": 0.0000899, "valor": 2911.12},
    {"nome": "TV 55", "espaco": 0.400, "valor": 4346.99},
    {"nome": "TV 50", "espaco": 0.290, "valor": 3999.90},
    {"nome": "TV 42", "espaco": 0.200, "valor": 2999.00},
    {"nome": "Notebook Dell", "espaco": 0.00350, "valor": 2499.90},
    {"nome": "Ventilador Panasonic", "espaco": 0.496, "valor": 199.90},
    {"nome": "Microondas Electrolux", "espaco": 0.0424, "valor": 308.66},
    {"nome": "Microondas LG", "espaco": 0.0544, "valor": 429.90},
    {"nome": "Microondas Panasonic", "espaco": 0.0319, "valor": 299.29},
    {"nome": "Geladeira Brastemp", "espaco": 0.635, "valor": 849.00},
    {"nome": "Geladeira Consul", "espaco": 0.870, "valor": 1199.89},
    {"nome": "Notebook Lenovo", "espaco": 0.498, "valor": 1999.90},
    {"nome": "Notebook Asus", "espaco": 0.527, "valor": 3999.00},
]

In [4]:
df_produtos = pd.DataFrame(produtos)
df_produtos.head()

,nome,espaco,valor
0,Geladeira,0.75100,999.90
1,Celular,0.00009,2911.12
2,TV 55,0.40000,4346.99
3,TV 50,0.29000,3999.90
4,TV 42,0.20000,2999.00


In [5]:
def avalicao(individuo):
    espaco_total = sum(df_produtos["espaco"] * individuo)
    valor_total = sum(df_produtos["valor"] * individuo)
    if espaco_total > limite:
        valor_total = 1
    return valor_total / 100000,

In [6]:
# setup do deap

# define o tipo fitness (min ou max) e o tipo do indivíduo
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# criação da toolbox
toolbox = base.Toolbox()

# função de inicialização dos indivíduos
toolbox.register("attr_bool", np.random.randint, 0, 2)
# registrando o indivíduo
toolbox.register(
    "individual",
    tools.initRepeat,
    creator.Individual,
    toolbox.attr_bool,
    n=df_produtos.shape[0],
)
# registrando a população
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# registrando a função de avaliação
toolbox.register("evaluate", avalicao)
# registrando a função de crossover
toolbox.register("mate", tools.cxOnePoint)
# registrando a função de mutação
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
# registrando a função de seleção
toolbox.register("select", tools.selRoulette)

In [7]:
if __name__ ==  "__main__":
    populacao = toolbox.population(n=100)

    CXPB, MUTPB, NGEN = 1.0, 0.05, 100
    estatisticas = tools.Statistics(key=lambda individuo: individuo.fitness.values)
    estatisticas.register("max", np.max)

    populacao, info = algorithms.eaSimple(
        population=populacao,
        toolbox=toolbox,
        cxpb=CXPB,
        mutpb=MUTPB,
        ngen=NGEN,
        stats=estatisticas,
        verbose=False,
    )

    populacao, info
    melhores = tools.selBest(populacao, k=1)
    for individuo in melhores:
        print(individuo)
        print(individuo.fitness)
        espaco_total = np.sum(df_produtos["espaco"] * individuo)
        valor_total = np.sum(df_produtos["valor"] * individuo)
        print(f"Espaço total: {espaco_total}")
        print(f"Valor total: {valor_total}")

        print("Produtos:" , ", ".join(df_produtos['nome'].loc[df_produtos.index[individuo] == 1].values))

[0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1]
(0.2306447,)
Espaço total: 1.9609899
Valor total: 23064.47
Produtos: Celular, TV 55, TV 50, TV 42, Notebook Dell, Microondas Electrolux, Notebook Lenovo, Notebook Asus
